In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from utils import session_config
os.environ['CUDA_VISIBLE_DEVICES'] = "1" # 사용 gpu 선택
session_config.setup_gpus(True, 0.5) # gpu 메모리 사용량 세팅

1 Physical GPUs, 1 Logical GPUs


In [3]:
%config Completer.use_jedi = False

In [4]:
import tensorflow as tf

# load your model already built at phase 1.
model = tf.keras.models.load_model("models/cifar_classifier.h5")

# load your dataset with tf.data
cifar10 = tf.keras.datasets.cifar10
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(buffer_size = 20000, reshuffle_each_iteration=False)
val_ds = train_ds.take(10000)
train_ds = train_ds.skip(10000)

# Hyper-parameter tuning

주요한 하이퍼파라미터는 다음과 같다. 

- optimizer = ["ADAM_W, "RMSPROP"]
- learning rate = [1E-3, 1E-4, 1E-5]
- learning rate scheduler = [constant, piecewise decay, linear decay, cosine decay restart]
- batch_size = [32, 64, 128]
- weight_decay = [1E-5, 5E-5, 1E-4]

리스트에 포함된 파라미터값에 의해 총 2x3x4x3x3 = 216가지 경우의 수에 대해 실험할 수 있다. 각 실험에 대해 Early stopping 지점까지 학습하고 validation 정확도를 측정한다. 우리는 주요한 하이퍼파라미터를 Grid search 방식으로 사전에 결정한다. 이런 방식은 하이퍼파라미터 튜닝에서 더 낮은 차원의 파라미터 공간에 대한 탐색을 수행할 수 있다. 이는 하이퍼파라미터 최적화에 대한 약간의 타협을 통해 효율적인 파라미터 튜닝을 가능하게 한다.

In [5]:
from utils.hparam_tune import Keras_Tuner

In [7]:
log_dir = "/home/files/AI_project_templates/logs/tuner_test3"
ckpt_dir = "/home/files/AI_project_templates/ckpts/tuner_test3"
initial_model_path = "models/cifar_classifier.h5"

tuner = Keras_Tuner(log_dir, ckpt_dir, initial_model_path, train_ds, val_ds, metric="accuracy")

In [ ]:
tuner.tuning()

--- Starting trial: run-0
{'init_lr': 1e-05, 'optimizer': 'ADAM_W', 'lr_scheduler': 'constant', 'batch_size': 32, 'weight_decay': 1e-05}
constant 1e-05
Epoch 1/300
1250/1250 [==============================] - 11s 6ms/step - loss: 1.6641 - acc: 0.4121 - val_loss: 1.3736 - val_acc: 0.5041

Epoch 00001: val_loss improved from inf to 1.37358, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-0/model-1.h5
Epoch 2/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.2351 - acc: 0.5601 - val_loss: 1.1268 - val_acc: 0.6039

Epoch 00002: val_loss improved from 1.37358 to 1.12682, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-0/model-2.h5
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.0549 - acc: 0.6252 - val_loss: 1.1148 - val_acc: 0.6030

Epoch 00003: val_loss improved from 1.12682 to 1.11478, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-0/model-3.h5
Epoch 4/300
1250/1250 [

constant 1e-05
Epoch 1/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.6665 - acc: 0.4060 - val_loss: 1.3440 - val_acc: 0.5058

Epoch 00001: val_loss improved from inf to 1.34401, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-2/model-1.h5
Epoch 2/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.2186 - acc: 0.5659 - val_loss: 1.1055 - val_acc: 0.6144

Epoch 00002: val_loss improved from 1.34401 to 1.10552, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-2/model-2.h5
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.0341 - acc: 0.6365 - val_loss: 1.0354 - val_acc: 0.6376

Epoch 00003: val_loss improved from 1.10552 to 1.03540, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-2/model-3.h5
Epoch 4/300
1250/1250 [==============================] - 6s 5ms/step - loss: 0.9161 - acc: 0.6788 - val_loss: 0.9813 - val_acc: 0.6633

Epoch 00004: val_loss imp


Epoch 00002: val_loss improved from 1.36699 to 1.16195, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-4/model-2.h5
Epoch 3/300
625/625 [==============================] - 3s 5ms/step - loss: 1.0234 - acc: 0.6394 - val_loss: 1.0335 - val_acc: 0.6421

Epoch 00003: val_loss improved from 1.16195 to 1.03346, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-4/model-3.h5
Epoch 4/300
625/625 [==============================] - 3s 5ms/step - loss: 0.9018 - acc: 0.6819 - val_loss: 0.9946 - val_acc: 0.6551

Epoch 00004: val_loss improved from 1.03346 to 0.99460, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-4/model-4.h5
Epoch 5/300
625/625 [==============================] - 3s 5ms/step - loss: 0.8135 - acc: 0.7150 - val_loss: 0.9718 - val_acc: 0.6681

Epoch 00005: val_loss improved from 0.99460 to 0.97176, saving model to /home/files/AI_project_templates/ckpts/tuner_test3/run-4/model-5.h5
Epoch 6/300
625/625 [================

In [ ]:

model = tf.keras.models.load_model("ckpts/...")
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_ds = self.test_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
model.evaluate(test_ds)